In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

In [2]:
# load the dataSets
data = pd.read_csv('flights_train.csv', index_col=0)
testData = pd.read_csv('flights_test.csv',index_col=0)
airportData = pd.read_csv('airports.csv', low_memory=False)

C:\Users\siebr\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [9]:
testModelData = data
# Add departure hour
testModelData['SCHEDULED_DEPARTURE'] = testModelData['SCHEDULED_DEPARTURE'].apply('{:0>4}'.format)
testModelData['DEPARTURE_HOUR'] = testModelData['SCHEDULED_DEPARTURE'].str[:2]
# Add arrival hour
testModelData['SCHEDULED_ARRIVAL'] = testModelData['SCHEDULED_ARRIVAL'].apply('{:0>4}'.format)
testModelData['ARRIVAL_HOUR'] = testModelData['SCHEDULED_ARRIVAL'].str[:2]

In [10]:
# create dummies
dummy_fields = ['ARRIVAL_HOUR', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'AIRLINE']
for each in dummy_fields:
    dummies = pd.get_dummies(testModelData[each], prefix=each, drop_first=False)
    testModelData = pd.concat([testModelData, dummies], axis=1)
testModelData.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN
id,,,,,,,,,,,,,,,,,,,,,
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,0005,...,0,0,0,0,0,0,0,0,0,0
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,0010,...,0,0,0,0,0,0,0,0,0,0
2,2015,1,1,4,US,840,N171US,SFO,CLT,0020,...,0,0,0,0,0,0,0,1,0,0
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,0020,...,0,0,0,0,0,0,0,0,0,0
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,0025,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# remove original columns
testModelData = testModelData.drop(dummy_fields, axis=1)

In [12]:
# Select columns
trainData3 = testModelData.drop(columns=['YEAR', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'DEPARTURE_TIME' , 'TAXI_OUT', 'AIR_TIME', 'DISTANCE'])

In [13]:
# ReduceDataSet
trainData3=trainData3.sample(frac=0.2,random_state=200)

# Split data
X = trainData3.drop(columns='ARRIVAL_DELAY')
Y = trainData3['ARRIVAL_DELAY']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(450968, 692) (112743, 692) (450968,) (112743,)


In [14]:
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0, n_jobs=-1)
regressor.fit(X_train, Y_train)
# R^2 scores
print('Train:', regressor.score(X_train, Y_train))
print('Test:', regressor.score(X_test, Y_test))

Train: 0.9859337872273712
Test: 0.9440314888620656


# TEST DATA

In [16]:
# Create dummies
dummy_fields = ['ARRIVAL_HOUR', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'AIRLINE']
for each in dummy_fields:
    dummies = pd.get_dummies(testData[each], prefix=each, drop_first=False)
    testData = pd.concat([testData, dummies], axis=1)
testData.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,AIRLINE_DL,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_VX,AIRLINE_WN
id,,,,,,,,,,,,,,,,,,,,,
0,2015,7,1,3,NK,298,N624NK,LAS,IAH,0003,...,0,0,0,0,0,1,0,0,0,0
1,2015,7,1,3,AA,1965,N507AY,SFO,CLT,0005,...,0,0,0,0,0,0,0,0,0,0
2,2015,7,1,3,DL,2020,N3759,LAX,MEM,0005,...,1,0,0,0,0,0,0,0,0,0
3,2015,7,1,3,NK,612,N629NK,LAS,MSP,0005,...,0,0,0,0,0,1,0,0,0,0
4,2015,7,1,3,UA,1044,N39418,ANC,ORD,0008,...,0,0,0,0,0,0,0,1,0,0


In [18]:
# remove original dummies
testData = testData.drop(dummy_fields, axis=1)

In [19]:
# Select data
testData = testData.drop(columns=['YEAR', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'DEPARTURE_TIME' , 'TAXI_OUT', 'AIR_TIME', 'DISTANCE'])
# select missing columns
missingCols = X_test.columns.difference(testData.columns)
# add zeros to missing columns
for col in missingCols:
    testData[col] = 0

In [20]:
finalPrediction = regressor.predict(testData)
finalDataset = pd.DataFrame(data=finalPrediction, columns=['ARRIVAL_DELAY'])

In [21]:
# save file
finalDataset.to_csv('prediction.csv', index_label='id')